In [1]:
# Import necessary modules and functions

import pandas as pd
import seaborn as sns
import numpy as np
import nltk
import spacy

import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 16, 8

from scipy.stats import norm
from scipy import stats

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
nlp = English()


import warnings
warnings.filterwarnings('ignore')

kaggledf = pd.read_csv('KS_train_data.csv', delimiter = ',')
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/successerviceofothers/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
#nltk.download('vader_lexicon', quiet=False) # you can download 1 time then comment out
from nltk.sentiment.vader import SentimentIntensityAnalyzer #verify download
vader_model = SentimentIntensityAnalyzer()

In [47]:
nlp = spacy.load('en_core_web_sm') # en_core_web_sm

In [22]:
pd.set_option("display.max_rows", 10, "display.max_columns", None)
#categorical ranges for goal and pledged, sentiment vader (numerical & categorical)  
#columns to be removed: url's, location, id,
#kaggledf.dropna(inplace=True)
kaggledf

,project_id,backers_count,blurb,category,converted_pledged_amount,country,created_at,currency,deadline,fx_rate,goal,launched_at,name,pledged,staff_pick,usd_pledged,location,funded,subcategory,project_url,reward_url,blurb_vector_column,blurb_0,blurb_1,blurb_2,blurb_3,blurb_4,blurb_5,blurb_6,blurb_7,blurb_8,blurb_9,blurb_10,blurb_11,blurb_12,blurb_13,blurb_14,blurb_15,blurb_16,blurb_17,blurb_18,blurb_19
0,KS_000000,80,I will be an artist-in-residence at Elsewhere ...,art,3596,US,1325980505,USD,1334866560,1.000000,2800.0,1332969260,Elsewhere Studios artist-in-residency program!,3596.00,False,3596.000000,"Paonia, CO",True,mixed media,https://www.kickstarter.com/projects/hilaryeme...,https://www.kickstarter.com/projects/hilaryeme...,"[0.32483795, 0.04421308, -0.3800838, 0.0162373...",0.324838,0.044213,-0.380084,0.016237,-0.051155,-0.208816,0.107672,-0.189545,-0.270467,-0.087818,0.250261,-0.102271,0.209568,0.128131,0.196993,0.056132,-0.044138,-0.094276,0.156708,-0.185284
1,KS_000001,82,1000 Artists is a public art-making installati...,art,4586,US,1330926084,USD,1336440145,1.000000,4500.0,1332984145,1000 Artists: Presidential Inauguration 2013,4586.00,False,4586.000000,"Washington, DC",True,public art,https://www.kickstarter.com/projects/17146650/...,https://www.kickstarter.com/projects/17146650/...,"[-0.123726286, 0.15019314, 0.064539276, 0.1707...",-0.123726,0.150193,0.064539,0.170718,-0.100426,-0.206194,-0.097930,0.035693,-0.456791,0.211481,0.179926,-0.143337,0.051441,-0.041513,0.131939,0.192242,0.081829,-0.105904,0.245299,-0.012601
2,KS_000002,21,"The Sequel to ‘My Favorite Machine”, ""MyPhoneH...",art,5217,US,1332382894,USD,1338175739,1.000000,5000.0,1332991739,MyPhoneHenge,5217.00,False,5217.000000,"Frisco, TX",True,sculpture,https://www.kickstarter.com/projects/belveal/m...,https://www.kickstarter.com/projects/belveal/m...,"[0.09615797, -0.08359938, -0.13714573, -0.1335...",0.096158,-0.083599,-0.137146,-0.133595,-0.092476,-0.279922,0.037794,0.183541,-0.543861,-0.093223,0.166298,0.095719,0.099650,0.126690,0.260994,0.038817,0.199036,-0.384883,-0.075385,-0.048125
3,KS_000003,37,A film exploring the role and value of art edu...,art,7160,GB,1332342451,USD,1334806096,1.000000,6500.0,1332991696,Walk-Through,7160.00,False,7160.000000,"Glasgow, UK",True,art,https://www.kickstarter.com/projects/107813091...,https://www.kickstarter.com/projects/107813091...,"[-0.10111227, -0.14367606, 0.14420073, -0.2878...",-0.101112,-0.143676,0.144201,-0.287835,-0.139570,0.081892,-0.177235,0.177225,-0.250890,0.130282,0.154965,-0.135298,0.066694,-0.049866,0.068505,-0.028555,0.068770,-0.120629,0.047248,0.012347
4,KS_000004,153,We need to build a kitchen for Habitable Space...,art,15445,US,1328562617,USD,1335584240,1.000000,15000.0,1332992240,A kitchen for Habitable Spaces,15445.00,False,15445.000000,"Kingsbury, TX",True,public art,https://www.kickstarter.com/projects/104409276...,https://www.kickstarter.com/projects/104409276...,"[-0.11558902, 0.07321389, 0.19051558, 0.181122...",-0.115589,0.073214,0.190516,0.181123,-0.042319,-0.220176,-0.054905,0.046121,-0.251400,0.117523,0.232130,-0.135302,0.094642,0.010051,0.119201,-0.092462,-0.024277,-0.096373,-0.137684,-0.314083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,KS_099995,104,Make the workshop/ small stage space at Jimmy'...,theater,7595,US,1348677893,USD,1356552252,1.000000,7000.0,1353960252,"Help Fund the ""Back Room"" Arts Space at Jimmy'...",7595.43,False,7595.430000,"New York, NY",True,spaces,https://www.kickstarter.com/projects/jimmycarb...,https://www.kickstarter.com/projects/jimmycarb...,"[0.12864462, -0.01705722, -0.14170773, -0.1966...",0.128645,-0.017057,-0.141708,-0.196699,-0.073584,-0.278953,0.117521,0.050178,-0.174476,-0.217674,0.186276,0.058369,0.071320,0.053186,0.205116,-0.007378,0.129437,-0.188045,-0.272203,-0.124158
99996,KS_099996,307,Help create a permane

In [7]:
def run_vader(textual_unit, 
              lemmatize=False, 
              parts_of_speech_to_consider=set(),
              verbose=0):
    """
    Run VADER on a sentence from spacy
    
    :param str textual unit: a textual unit, e.g., sentence, sentences (one string)
    (by looping over doc.sents)
    :param bool lemmatize: If True, provide lemmas to VADER instead of words
    :param set parts_of_speech_to_consider:
    -empty set -> all parts of speech are provided
    -non-empty set: only these parts of speech are considered
    :param int verbose: if set to 1, information is printed
    about input and output
    
    :rtype: dict
    :return: vader output dict
    """
    doc = nlp(textual_unit)
        
    input_to_vader = []

    for sent in doc.sents:
        for token in sent:

            to_add = token.text

            if lemmatize:
                to_add = token.lemma_

                if to_add == '-PRON-': 
                    to_add = token.text

            if parts_of_speech_to_consider:
                if token.pos_ in parts_of_speech_to_consider:
                    input_to_vader.append(to_add) 
            else:
                input_to_vader.append(to_add)

    scores = vader_model.polarity_scores(' '.join(input_to_vader))
    
    if verbose >= 1:
        print()
        print('INPUT SENTENCE', sent)
        print('INPUT TO VADER', input_to_vader)
        print('VADER OUTPUT', scores)

    return scores

In [8]:
sentences = ["Here are my sentences.",
             "It's a nice day.",
             "It's a rainy day."]

for sent in sentences:
    scores = vader_model.polarity_scores(sent)
    print()
    print('INPUT SENTENCE', sent)
    print('VADER OUTPUT', scores)


INPUT SENTENCE Here are my sentences.
VADER OUTPUT {'neg': 0.0, 'neu': 0.714, 'pos': 0.286, 'compound': 0.0516}

INPUT SENTENCE It's a nice day.
VADER OUTPUT {'neg': 0.0, 'neu': 0.417, 'pos': 0.583, 'compound': 0.4215}

INPUT SENTENCE It's a rainy day.
VADER OUTPUT {'neg': 0.394, 'neu': 0.606, 'pos': 0.0, 'compound': -0.0772}


In [37]:
run_vader(sentences[2], lemmatize=True)

{'neg': 0.302, 'neu': 0.698, 'pos': 0.0, 'compound': -0.0772}

In [38]:
sent_input = kaggledf['blurb'][5]
print(sent_input)

Source Material is 2 person exhibition of the paintings of Brian Dupont and Chris Rusak at the Skydive Artspace in Houston, Texas.


In [39]:
run_vader(kaggledf['blurb'][56], lemmatize=True)


{'neg': 0.0, 'neu': 0.759, 'pos': 0.241, 'compound': 0.6908}

In [40]:
kaggledf['blurb']

0        I will be an artist-in-residence at Elsewhere ...
1        1000 Artists is a public art-making installati...
2        The Sequel to ‘My Favorite Machine”, "MyPhoneH...
3        A film exploring the role and value of art edu...
4        We need to build a kitchen for Habitable Space...
                               ...                        
99995    Make the workshop/ small stage space at Jimmy'...
99996    Help create a permanent home for live comedy s...
99997    Voix de Ville is a pop-up imaginarium of neo-v...
99998    Help Synetic Theater create a new Studio to pr...
99999    Be part of building Cardiff's first pub theatr...
Name: blurb, Length: 99995, dtype: object

In [9]:
sentiment_scores = []
for sentence in kaggledf['blurb']:
    sentiment_scores.append(run_vader(sentence, lemmatize=True)['compound'])

In [ ]:
kaggledf['blurb_sentiment_scores'] = sentiment_scores

In [88]:
kaggledf['blurb_sentiment_scores']

0        0.0000
1        0.0000
2        0.7717
3        0.3400
4        0.0000
          ...  
99995   -0.1007
99996    0.7430
99997    0.4199
99998    0.9184
99999    0.0000
Name: blurb_sentiment_scores, Length: 99995, dtype: float64

In [49]:
tokenized_blurb_list = []
for blurb in kaggledf['blurb']:
    try:
        
        tokenized_blurb_list.append(nltk.word_tokenize(blurb))
    except:
        print(blurb)

In [50]:
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(tokenized_blurb_list)]


In [51]:
model = Doc2Vec(tagged_data, vector_size = 10, window = 2, min_count = 2, epochs = 10)


In [9]:
blurb_vector_list = []
for blurb in tokenized_blurb_list:
    blurb_vector_list.append(model.infer_vector(blurb))

In [11]:
kaggledf['blurb_vector_column'] = blurb_vector_list

In [16]:
l_of_l = [[] for _ in range(20)]
print(l_of_l)

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]


In [17]:
for blurb_vector in blurb_vector_list:
    for i, element in enumerate(blurb_vector):
        l_of_l[i].append(element)
        

In [20]:
for i, column in enumerate(l_of_l):
    name = "blurb_" + str(i)
    kaggledf[name] = column

In [25]:
kaggledf.drop(columns=['blurb_vector_column'], inplace=True)

In [52]:
tokenized_name_list = []
for name in kaggledf['name']:
    tokenized_name_list.append(nltk.word_tokenize(name))


In [53]:
tagged_name_list = [TaggedDocument(d, [i]) for i, d in enumerate(tokenized_name_list)]

In [54]:
name_model = Doc2Vec(tagged_name_list, vector_size = 10, window = 2, min_count = 1, epochs = 10)

In [55]:
name_vector_list = []
for name in tokenized_name_list:
    name_vector_list.append(model.infer_vector(name))

In [62]:
split_name_vectors = [[] for _ in range(10)]
for name_vector in name_vector_list:
    for i, element in enumerate(name_vector):
        split_name_vectors[i].append(element)
        


In [63]:
for i, column in enumerate(split_name_vectors):
    name = "name_" + str(i)
    kaggledf[name] = column

,project_id,backers_count,blurb,category,converted_pledged_amount,country,created_at,currency,deadline,fx_rate,goal,launched_at,name,pledged,staff_pick,usd_pledged,location,funded,subcategory,project_url,reward_url,blurb_0,blurb_1,blurb_2,blurb_3,blurb_4,blurb_5,blurb_6,blurb_7,blurb_8,blurb_9,blurb_10,blurb_11,blurb_12,blurb_13,blurb_14,blurb_15,blurb_16,blurb_17,blurb_18,blurb_19,name_0,name_1,name_2,name_3,name_4,name_5,name_6,name_7,name_8,name_9
0,KS_000000,80,I will be an artist-in-residence at Elsewhere ...,art,3596,US,1325980505,USD,1334866560,1.000000,2800.0,1332969260,Elsewhere Studios artist-in-residency program!,3596.00,False,3596.000000,"Paonia, CO",True,mixed media,https://www.kickstarter.com/projects/hilaryeme...,https://www.kickstarter.com/projects/hilaryeme...,0.324838,0.044213,-0.380084,0.016237,-0.051155,-0.208816,0.107672,-0.189545,-0.270467,-0.087818,0.250261,-0.102271,0.209568,0.128131,0.196993,0.056132,-0.044138,-0.094276,0.156708,-0.185284,-0.108196,-0.134751,0.049863,-0.304463,-0.080544,-0.006730,0.012342,-0.009126,-0.066908,-0.058336
1,KS_000001,82,1000 Artists is a public art-making installati...,art,4586,US,1330926084,USD,1336440145,1.000000,4500.0,1332984145,1000 Artists: Presidential Inauguration 2013,4586.00,False,4586.000000,"Washington, DC",True,public art,https://www.kickstarter.com/projects/17146650/...,https://www.kickstarter.com/projects/17146650/...,-0.123726,0.150193,0.064539,0.170718,-0.100426,-0.206194,-0.097930,0.035693,-0.456791,0.211481,0.179926,-0.143337,0.051441,-0.041513,0.131939,0.192242,0.081829,-0.105904,0.245299,-0.012601,-0.076324,-0.033846,0.136053,-0.215700,0.072207,-0.023868,0.199541,-0.100112,-0.379975,-0.107390
2,KS_000002,21,"The Sequel to ‘My Favorite Machine”, ""MyPhoneH...",art,5217,US,1332382894,USD,1338175739,1.000000,5000.0,1332991739,MyPhoneHenge,5217.00,False,5217.000000,"Frisco, TX",True,sculpture,https://www.kickstarter.com/projects/belveal/m...,https://www.kickstarter.com/projects/belveal/m...,0.096158,-0.083599,-0.137146,-0.133595,-0.092476,-0.279922,0.037794,0.183541,-0.543861,-0.093223,0.166298,0.095719,0.099650,0.126690,0.260994,0.038817,0.199036,-0.384883,-0.075385,-0.048125,0.065590,0.019809,0.102891,-0.209886,0.026325,0.060591,0.148444,0.018075,-0.232398,0.003528
3,KS_000003,37,A film exploring the role and value of art edu...,art,7160,GB,1332342451,USD,1334806096,1.000000,6500.0,1332991696,Walk-Through,7160.00,False,7160.000000,"Glasgow, UK",True,art,https://www.kickstarter.com/projects/107813091...,https://www.kickstarter.com/projects/107813091...,-0.101112,-0.143676,0.144201,-0.287835,-0.139570,0.081892,-0.177235,0.177225,-0.250890,0.130282,0.154965,-0.135298,0.066694,-0.049866,0.068505,-0.028555,0.068770,-0.120629,0.047248,0.012347,-0.020933,-0.019340,-0.042779,0.016940,-0.020686,-0.039864,-0.047999,-0.034878,0.040584,0.026431
4,KS_000004,153,We need to build a kitchen for Habitable Space...,art,15445,US,1328562617,USD,1335584240,1.000000,15000.0,1332992240,A kitchen for Habitable Spaces,15445.00,False,15445.000000,"Kingsbury, TX",True,public art,https://www.kickstarter.com/projects/104409276...,https://www.kickstarter.com/projects/104409276...,-0.115589,0.073214,0.190516,0.181123,-0.042319,-0.220176,-0.054905,0.046121,-0.251400,0.117523,0.232130,-0.135302,0.094642,0.010051,0.119201,-0.092462,-0.024277,-0.096373,-0.137684,-0.314083,-0.107394,0.069591,0.044209,-0.052610,0.072628,0.036010,0.045027,0.036639,-0.032606,0.025273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,KS_099995,104,Make the workshop/ small stage space at Jimmy'...,theater,7595,US,1348677893,USD,1356552252,1.000000,7000.0,1353960252,"Help Fund the ""Back Room"" Arts Space at Jimmy'...",7595.43,False,7595.430000,"New York, NY",True,spaces,https://www.kickstarter.com/projects/jimmycarb...,https://www.kickstarter.com/pro